In [51]:
from sklearn import metrics
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from catboost import CatBoostClassifier

In [52]:
df=pd.read_csv('tel_churn.csv')

In [53]:
df.head()

,Unnamed: 0,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,0,29.85,29.85,0,1,0,0,1,1,...,0,0,1,0,1,0,0,0,0,0
1,1,0,56.95,1889.50,0,0,1,1,0,1,...,0,0,0,1,0,0,1,0,0,0
2,2,0,53.85,108.15,1,0,1,1,0,1,...,0,0,0,1,1,0,0,0,0,0
3,3,0,42.30,1840.75,0,0,1,1,0,1,...,1,0,0,0,0,0,0,1,0,0
4,4,0,70.70,151.65,1,1,0,1,0,1,...,0,0,1,0,1,0,0,0,0,0


In [54]:
df=df.drop('Unnamed: 0',axis=1)

In [55]:
df.shape

(7032, 51)

In [56]:
x=df.drop('Churn',axis=1)
x.head()

,SeniorCitizen,MonthlyCharges,TotalCharges,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,29.85,29.85,1,0,0,1,1,0,1,...,0,0,1,0,1,0,0,0,0,0
1,0,56.95,1889.50,0,1,1,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
2,0,53.85,108.15,0,1,1,0,1,0,0,...,0,0,0,1,1,0,0,0,0,0
3,0,42.30,1840.75,0,1,1,0,1,0,1,...,1,0,0,0,0,0,0,1,0,0
4,0,70.70,151.65,1,0,1,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [57]:
y=df['Churn']

In [58]:
y.value_counts().sum()

7032

In [59]:
y[:5]

0    0
1    0
2    1
3    0
4    1
Name: Churn, dtype: int64

In [60]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [61]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((4922, 50), (2110, 50), (4922,), (2110,))

In [62]:
y_train.value_counts()

Churn
0    3614
1    1308
Name: count, dtype: int64

In [63]:
sm = SMOTE()
x_resampled, y_resampled = sm.fit_resample(x_train, y_train)

In [64]:
y_resampled.value_counts()

Churn
1    3614
0    3614
Name: count, dtype: int64

In [65]:
x_resampled.shape,y_resampled.shape

((7228, 50), (7228,))

In [66]:
scaler = MinMaxScaler()
x_resampled = scaler.fit_transform(x_resampled)
x_test = scaler.transform(x_test)
x_resampled = pd.DataFrame(x_resampled, columns=x.columns)
x_test = pd.DataFrame(x_test, columns=x.columns)

In [67]:
x_resampled.head()

,SeniorCitizen,MonthlyCharges,TotalCharges,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0.0,0.401096,0.060120,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.431490,0.180373,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.515197,0.023263,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.551570,0.057830,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.420030,0.454059,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [68]:
param_grids = {
    'Logistic Regression': {
        'C': [0.01, 0.1, 1, 10, 100],
        'solver': ['liblinear', 'saga']
    },
    'Decision Tree': {
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'Random Forest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10]
    },
    'XGBoost': {
        'n_estimators': [50, 100, 200,300],
        'learning_rate': [0.001, 0.01, 0.2],
        'max_depth': [3, 5, 7,9],
        'subsample': [0.8, 1.0]
    },
    'CatBoost': {
        'iterations': [50, 100],
        'learning_rate': [0.01, 0.1],
        'depth': [4, 6, 8]
    }
}

In [69]:
# Initialize the models
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'XGBoost': xgb.XGBClassifier(),
    'Naive Bayes': GaussianNB(),
    'CatBoost': CatBoostClassifier()
}

In [70]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

best_models = {}
best_scores = []

# Loop through each model
for model_name, model in models.items():
    if model_name == 'Naive Bayes':
        model.fit(x_resampled, y_resampled)
        y_pred = model.predict(x_test)
        
        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred, average='binary')
        f1 = f1_score(y_test, y_pred, average='binary')
        precision = precision_score(y_test, y_pred, average='binary')
        
        # Store the best model and its scores
        best_models[model_name] = model
        best_scores.append({
            'Model': model_name,
            'Accuracy': accuracy,
            'Recall': recall,
            'F1 Score': f1,
            'Precision': precision
        })
        print(f'{model_name} Accuracy: {accuracy:.2f}')
        print(f'{model_name} Recall: {recall:.2f}')
        print(f'{model_name} F1 Score: {f1:.2f}')
        print(f'{model_name} Precision: {precision:.2f}\n')
    else:
        print(f'Tuning hyperparameters for {model_name}...')
        grid_search = GridSearchCV(model, param_grids[model_name], cv=5, n_jobs=-1, verbose=1)
        grid_search.fit(x_resampled, y_resampled)
        
        # Get the best model from GridSearchCV
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(x_test)
        
        # Calculate evaluation metrics
        accuracy = accuracy_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred, average='binary')
        f1 = f1_score(y_test, y_pred, average='binary')
        precision = precision_score(y_test, y_pred, average='binary')
        
        # Store the best model and its scores
        best_models[model_name] = best_model
        best_scores.append({
            'Model': model_name,
            'Accuracy': grid_search.best_score_,
            'Recall': recall,
            'F1 Score': f1,
            'Precision': precision
        })
        
        print(f'Best parameters for {model_name}: {grid_search.best_params_}')
        print(f'Best cross-validation score: {grid_search.best_score_:.2f}')
        print(f'{model_name} Test Accuracy: {accuracy:.2f}')
        print(f'{model_name} Test Recall: {recall:.2f}')
        print(f'{model_name} Test F1 Score: {f1:.2f}')
        print(f'{model_name} Test Precision: {precision:.2f}\n')

# Create a DataFrame to display all results
results_df = pd.DataFrame(best_scores)


Tuning hyperparameters for Logistic Regression...
Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\Nimesh_140588\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best parameters for Logistic Regression: {'C': 1, 'solver': 'saga'}
Best cross-validation score: 0.83
Logistic Regression Test Accuracy: 0.79
Logistic Regression Test Recall: 0.52
Logistic Regression Test F1 Score: 0.57
Logistic Regression Test Precision: 0.64

Tuning hyperparameters for Decision Tree...
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters for Decision Tree: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2}
Best cross-validation score: 0.82
Decision Tree Test Accuracy: 0.75
Decision Tree Test Recall: 0.61
Decision Tree Test F1 Score: 0.56
Decision Tree Test Precision: 0.52

Tuning hyperparameters for Random Forest...
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters for Random Forest: {'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 200}
Best cross-validation score: 0.85
Random Forest Test Accuracy: 0.78
Random Forest Test Recall: 0.56
Random Forest Test F1 Score: 0.58
Random Forest Test Prec

In [71]:
results_df

,Model,Accuracy,Recall,F1 Score,Precision
0,Logistic Regression,0.833727,0.520499,0.573111,0.637555
1,Decision Tree,0.816566,0.606061,0.561056,0.522273
2,Random Forest,0.848666,0.559715,0.579336,0.600382
3,XGBoost,0.847558,0.631016,0.610871,0.591973
4,Naive Bayes,0.740284,0.778966,0.614627,0.507549
5,CatBoost,0.845346,0.588235,0.596745,0.605505


In [77]:
y_pred=best_model.predict(x_test)

In [78]:
metrics.confusion_matrix(y_test, y_pred)

array([[1334,  215],
       [ 231,  330]], dtype=int64)

In [79]:
import pickle
filename = 'model.sav'
pickle.dump(best_model, open(filename, 'wb'))